In [1]:
import gc
import pandas as pd
import numpy as np
import os

from tqdm import tqdm_notebook

pd.set_option('display.max_columns', 100)

from utils import loadpkl, to_feature, line_notify

%matplotlib inline

In [13]:
# load pkls
df_sales = loadpkl('../feats/sales.pkl')
df_calendar = loadpkl('../feats/calendar.pkl')
df_sell_prices = loadpkl('../feats/sell_prices.pkl')
df_sub = pd.read_csv('../input/sample_submission.csv')

In [15]:
df_sales.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,is_test
60034805,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1969,0,1
60034806,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1969,0,1
60034807,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1969,0,1
60034808,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1969,0,1
60034809,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1969,0,1


In [17]:
df_calendar.tail()

,date,wm_yr_wk,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,seasonality
1964,2016-06-15,11620,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1,0.962513
1965,2016-06-16,11620,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0,0.967028
1966,2016-06-17,11620,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0,0.971257
1967,2016-06-18,11621,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0,0.975200
1968,2016-06-19,11621,2,6,2016,d_1969,13.0,0.0,3.0,0.0,0,0,0,0.978856


In [5]:
df_sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [6]:
df_sub.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60979,FOODS_3_827_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
# merge
df_sales = df_sales.merge(df_calendar, on='d',how='left')
df_sales = df_sales.merge(df_sell_prices, on=['store_id','item_id','wm_yr_wk'],how='left')
df_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,is_test,date,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,seasonality,sell_price
0,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,0,2011-01-29,11101,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,-0.878612,0.46
1,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2,0,2011-01-29,11101,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,-0.878612,1.56
2,HOBBIES_1_015_CA_1_validation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,d_1,4,0,2011-01-29,11101,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,-0.878612,0.70
3,HOBBIES_1_016_CA_1_validation,HOBBIES_1_016,HOBBIES_1,HOBBIES,CA_1,CA,d_1,5,0,2011-01-29,11101,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,-0.878612,0.70
4,HOBBIES_1_022_CA_1_validation,HOBBIES_1_022,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2,0,2011-01-29,11101,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,-0.878612,6.86


In [2]:
df_calendar = loadpkl('../feats/calendar.pkl')

In [4]:
df_calendar.date > 

0      2011-01-29
1      2011-01-30
2      2011-01-31
3      2011-02-01
4      2011-02-02
5      2011-02-03
6      2011-02-04
7      2011-02-05
8      2011-02-06
9      2011-02-07
10     2011-02-08
11     2011-02-09
12     2011-02-10
13     2011-02-11
14     2011-02-12
15     2011-02-13
16     2011-02-14
17     2011-02-15
18     2011-02-16
19     2011-02-17
20     2011-02-18
21     2011-02-19
22     2011-02-20
23     2011-02-21
24     2011-02-22
25     2011-02-23
26     2011-02-24
27     2011-02-25
28     2011-02-26
29     2011-02-27
          ...    
1939   2016-05-21
1940   2016-05-22
1941   2016-05-23
1942   2016-05-24
1943   2016-05-25
1944   2016-05-26
1945   2016-05-27
1946   2016-05-28
1947   2016-05-29
1948   2016-05-30
1949   2016-05-31
1950   2016-06-01
1951   2016-06-02
1952   2016-06-03
1953   2016-06-04
1954   2016-06-05
1955   2016-06-06
1956   2016-06-07
1957   2016-06-08
1958   2016-06-09
1959   2016-06-10
1960   2016-06-11
1961   2016-06-12
1962   2016-06-13
1963   201